<a href="https://colab.research.google.com/github/PolChip/2course_mash_lear/blob/main/lab_2_mash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests # Для отправки HTTP-запросов к сайту
from bs4 import BeautifulSoup # Для парсинга HTML и извлечения данных
import pandas as pd

def parse_news(news_count=10): # Берем кол-во новостей(по умолчанию 10) и парсим главную страницу
    url = "https://lenta.ru/"

    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_list = []

        # Ищем все новостные блоки
        news_items = soup.find_all('a', class_=lambda x: x and ('card' in x or 'toptitle' in x))

        for item in news_items:
            if len(news_list) >= news_count:
                break

            try:
                # Извлекаем заголовок
                title_elem = item.find(['h3', 'span'])
                if title_elem:
                    title = title_elem.get_text(strip=True)

                    # Извлекаем ссылку
                    link = item.get('href')
                    if link and not link.startswith('http'):
                        link = 'https://lenta.ru' + link

                    # Парсим полный текст новости
                    full_text = parse_full_text(link) if link else ""

                    news_list.append({
                        'title': title,
                        'link': link,
                        'full_text': full_text
                    })

            except Exception as e:
                continue

        return pd.DataFrame(news_list)

    except Exception as e:
        print(f"Ошибка: {e}")
        return pd.DataFrame()

def parse_full_text(url):
    """Парсит полный текст новости по ссылке"""
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Ищем текст новости в разных возможных контейнерах
        text_blocks = soup.find_all('p', class_=lambda x: x and 'body' in x)
        if not text_blocks:
            text_blocks = soup.find_all('div', class_=lambda x: x and 'body' in x)

        full_text = ' '.join([block.get_text(strip=True) for block in text_blocks])
        return full_text[:500]  # Ограничиваем длину текста

    except:
        return ""

# Основная программа
if __name__ == "__main__":
    # Ввод количества новостей с клавиатуры
    while True:
        try:
            count = int(input("Введите количество новостей для парсинга: "))
            if count > 0:
                break
            else:
                print("Число должно быть больше 0!")
        except ValueError:
            print("Пожалуйста, введите целое число!")

    df = parse_news(count)

    # Выводим результаты
    print(f"\nУспешно собрано {len(df)} новостей:")
    print("=" * 50)

    for i, row in df.iterrows():
        print(f"{i+1}. {row['title']}")
        if row['full_text']:
            print(f"   Текст: {row['full_text'][:100]}...")
        print()

    # Сохраняем в CSV
    df.to_csv('news.csv', index=False, encoding='utf-8')
    print(f"Данные сохранены в файл 'news.csv'")

Введите количество новостей для парсинга: 5

Успешно собрано 5 новостей:
1. Один налог повысят в нескольких регионах России. Власти объяснили это тем, что население богатеет
   Текст: Губернаторы трех российских регионов — Новосибирской и Оренбургской областей и Красноярского края — ...

2. Стало известно о возможной отставке генерала из переговорной группы России в Стамбуле
   Текст: Член переговорной группы поУкраиневСтамбулегенерал-полковникАлександр Фоминможет уйти в отставку с п...

3. Российский пенсионер решил заживо сжечь возлюбленную на глазах у внука
   Текст: ВГородцезадержан 70-летний местный житель, решивший заживо сжечь возлюбленную на глазах у внука. Об ...

4. Россиянин отпинал утопившего кота абьюзера
   Текст: ВКаменске-Уральскомсуд вынес приговор местному жителю, который встал на защиту сестры от сожителя-аб...

5. Предсказано возможное решение ЦБ по ключевой ставке в России
   Текст: На заседании в следующую пятницу, 24 октября, Центральный банк России (ЦБ) вероятне

In [6]:
import requests  # для отправки HTTP-запросов
from bs4 import BeautifulSoup  # для парсинга HTML
import pandas as pd  # для работы с таблицами данных

# Функция парсинга новостей
def parse_news(news_count=10):
    url = "https://lenta.ru/"

    try:
        response = requests.get(url) # Отправляем запрос к сайту
        soup = BeautifulSoup(response.content, 'html.parser') # Создаем объект для парсинга HTML
        news_list = []

        # Ищем все новостные блоки по определенным классам (карточек новостей и заголовков главных новостей)
        news_items = soup.find_all('a', class_=lambda x: x and ('card' in x or 'toptitle' in x))

        # Перебираем найденные новостные блоки
        for item in news_items:
            if len(news_list) >= news_count: # Проверяем, не собрали ли уже нужное количество новостей
                break

            try:
                # Извлекаем заголовок новости
                title_elem = item.find(['h3', 'span'])  # ищем в тегах h3 или span (заголовки новостей в своей HTML-структуре)
                if title_elem:  # если заголовок найден
                    title = title_elem.get_text(strip=True)  # извлекаем текст без пробелов

                    # Извлекаем ссылку на новость
                    link = item.get('href')  # получаем атрибут href
                    # Если ссылка относительная, делаем ее абсолютной
                    if link and not link.startswith('http'):
                        link = 'https://lenta.ru' + link

                    # Парсим полный текст новости по ссылке
                    full_text = parse_full_text(link) if link else ""

                    # Добавляем данные новости в список
                    news_list.append({
                        'title': title,  # заголовок
                        'link': link,  # ссылка
                        'full_text': full_text  # полный текст
                    })

            except Exception as e:  # если ошибка при обработке одной новости
                continue  # пропускаем эту новость и идем к следующей

        # Преобразуем список в DataFrame и возвращаем
        return pd.DataFrame(news_list)

    except Exception as e:  # если ошибка на уровне всей функции
        print(f"Ошибка: {e}")  # выводим сообщение об ошибке
        return pd.DataFrame()  # возвращаем пустой DataFrame

# Функция для парсинга полного текста новости
def parse_full_text(url):
    # Парсим полный текст новости по ссылке
    try:
        response = requests.get(url, timeout=5) # Отправляем запрос к странице новости с таймингом 5 секунд
        soup = BeautifulSoup(response.content, 'html.parser') # Парсим HTML страницы новости

        # Ищем текстовые блоки новости
        text_blocks = soup.find_all('p', class_=lambda x: x and 'body' in x)  # ищем параграфы с классом содержащим 'body'
        # Если не нашли параграфы, ищем в div
        if not text_blocks:
            text_blocks = soup.find_all('div', class_=lambda x: x and 'body' in x)

        # Объединяем текст из всех блоков в одну строку
        full_text = ' '.join([block.get_text(strip=True) for block in text_blocks])
        return full_text[:500]  # возвращаем первые 500 символов текста

    except:  # если любая ошибка при парсинге текста
        return ""  # возвращаем пустую строку

# Основная программа (запускается только при прямом запуске файла)
if __name__ == "__main__":
    # Ввод количества новостей с клавиатуры с проверкой
    while True:  # бесконечный цикл для ввода
        try:
            # Запрашиваем ввод числа от пользователя
            count = int(input("Введите количество новостей для парсинга: "))
            # Проверяем что число положительное
            if count > 0:
                break  # выходим из цикла если число корректное
            else:
                print("Число должно быть больше 0!")  # сообщение об ошибке
        except ValueError:  # если введено не число
            print("Пожалуйста, введите целое число!")  # сообщение об ошибке

    df = parse_news(count)  #парсим новости и сохраняем результат в переменную df

    # Выводим результаты парсинга
    print(f"\nУспешно собрано {len(df)} новостей:")  # количество собранных новостей

    # Перебираем все новости в DataFrame
    for i, row in df.iterrows():  # iterrows() возвращает индекс и строку
        print(f"{i+1}. {row['title']}")  # выводим номер и заголовок новости
        # Если есть полный текст новости
        if row['full_text']:
            print(f"   Текст: {row['full_text'][:100]}...")  # выводим первые 100 символов
        print()  # пустая строка для разделения

    # Сохраняем данные в CSV файл
    df.to_csv('news.csv', index=False, encoding='utf-8')  # сохраняем без индексов в UTF-8
    print(f"Данные сохранены в файл 'news.csv'")  # сообщение об успешном сохранении

Введите количество новостей для парсинга: 10

Успешно собрано 10 новостей:
1. На Украине признали потери после удара по учебному центру ВСУ
   Текст: Вооруженные силы Украины (ВСУ) понесли потери после удара по учебному центру. Об этом стало известно...

2. В США признали отставание от России в области обороны
   Текст: Армия США отстает от Вооруженных сил (ВС) РФ в области боевых беспилотников и стремится это исправит...

3. Раскрыт гонорар Маэстро Понасенкова за границей
   Текст: Историк и публицистЕвгений Понасенков(внесенМинюстом РФв реестр иноагентов), также известный как Маэ...

4. Турист получил ножевое ранение во время ограбления в Европе и не выжил
   Текст: Турист изСШАполучил ножевое ранение во время ограбления в Европе и не выжил. Об этомсообщаетCNN. Уто...

5. Бывшего охранника осудили за шпионаж
   Текст: Экс-охранника посольстваСШАвНорвегииосудили за шпионаж в пользу России иИрана. Об этомсообщаетпортал...

6. Борис Джонсон похвастался комплиментами от искусственного ин